In [2]:
import numpy as np
import pandas as pd 
from IPython.display import display

%matplotlib inline

train_data = pd.read_csv("final_training_data.csv")
test_data = pd.read_csv("test.csv")

In [3]:

pd.options.display.max_rows = 200
train_data.head()

,trip_duration,distance,pickup_c_0,pickup_c_1,pickup_c_2,pickup_c_3,pickup_c_4,pickup_c_5,pickup_c_6,pickup_c_7,...,passengers_2,passengers_3,passengers_4,passengers_5,passengers_6,passengers_7,passengers_8,passengers_9,vendor_1,vendor_2
0,455,1.502172,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,663,1.808660,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2124,6.379687,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,429,1.483632,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,435,1.187038,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
target = train_data['trip_duration']
train_data = train_data.drop(['trip_duration'],axis=1)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(train_data,target,random_state=0, test_size = 0.2)

In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1166915, 103)
(1166915,)
(291729, 103)
(291729,)


In [7]:
def rmsle(predicted,real):
    sum=0.0
    for x in range(len(predicted)):
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.5

In [8]:
from sklearn.linear_model import SGDRegressor

model = SGDRegressor()
model.fit(X_train, y_train)

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

In [9]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


pred = model.predict(X_test)
pred = np.abs(pred)

print(rmsle(pred, y_test.values))
print(mean_squared_error(y_test, pred))
print(r2_score(y_test,pred))

0.738256017107
52242774.7218
0.00427096567325


In [10]:
import keras

Using TensorFlow backend.


In [34]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=25, min_samples_leaf=25, min_samples_split=25)

regressor.fit(X_train,y_train)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=25,
           min_samples_split=25, min_weight_fraction_leaf=0.0,
           n_estimators=25, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [32]:
pred = regressor.predict(X_test)
pred = np.abs(pred)

print(rmsle(pred, y_test.values))
print(mean_squared_error(y_test, pred))
print(r2_score(y_test,pred))

0.635733652088
108262057.784
-1.06343699823


In [22]:
def fit_model(X, y):
    
    # Create cross-validation sets from the training data
    # sklearn version 0.18: ShuffleSplit(n_splits=10, test_size=0.1, train_size=None, random_state=None)
    # sklearn versiin 0.17: ShuffleSplit(n, n_iter=10, test_size=0.1, train_size=None, random_state=None)
    cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.1, random_state = 0)

    # TODO: Create a decision tree regressor object
    regressor =DecisionTreeRegressor()

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'max_depth':[1,2,3,4,5,6,7,8,9,10]}

    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(r2_score)

    # TODO: Create the grid search cv object --> GridSearchCV()
    # Make sure to include the right parameters in the object:
    # (estimator, param_grid, scoring, cv) which have values 'regressor', 'params', 'scoring_fnc', and 'cv_sets' respectively.
    grid = GridSearchCV(regressor,params,scoring_fnc,cv=cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_